## Week 1. Trapped Ions - Additional Task 2
Author: QuNovaComputing, Inc.
- - -

Calculate the fidelity from the experimental data from the [dataset](https://datadryad.org/stash/dataset/doi:10.5061/dryad.k6t1rj8).


First, extract the downloaded file in `./dataset/` and then, run `./dataset/extract.sh` to extract the tar.gz files.

Start from the importings :

In [8]:
from itertools import chain
from cirq import Simulator
from cirq.contrib.qasm_import import circuit_from_qasm
import os
import re
import csv

from Week1_Trapped_Ions.utils import get_xeb_from_hist_large

Check the `./dataset/README.md` for the information of the parameters, `n`, `m`, `s`, `e`, and `p`.
Put the desired parameters to the dictionary attributes of `ex_params` below.

In [9]:
ex_params={
    'n': 16,
    'm': 14,
    's': 0,
    'e': 0,
    'p': 'EFGH'
}

The below code finds the files for qasm for the circuit and experimental result.

In [10]:
ex_dir = f"./dataset/n{ex_params['n']}_m{ex_params['m']}"
ex_qasm = f"circuit_n{ex_params['n']}_m{ex_params['m']}_s{ex_params['s']}_e{ex_params['e']}_p{ex_params['p']}.qasm"
ex_meas = f"measurements_n{ex_params['n']}_m{ex_params['m']}_s{ex_params['s']}_e{ex_params['e']}_p{ex_params['p']}.txt"
print(f"Directory exists : {os.path.isdir(ex_dir)}")
print(f"Cirq exists : {os.path.isfile(os.path.join(ex_dir, ex_qasm))}")
print(f"Meas exists : {os.path.isfile(os.path.join(ex_dir, ex_meas))}")
if not (os.path.isdir(ex_dir)
        or os.path.isfile(os.path.join(ex_dir, ex_qasm))
        or os.path.isfile(os.path.join(ex_dir, ex_meas))):
    raise ValueError

Directory exists : True
Cirq exists : True
Meas exists : True


Reading the experimental result file, generate histogram.

In [11]:
num_shots = 0
meas_histogram = dict()
with open(os.path.join(ex_dir, ex_meas)) as of:
    while True:
        bin_str = of.readline().replace('\n', '')
        if len(bin_str) < 1:
            break
        if len(bin_str) != ex_params['n']:
            raise ValueError
        if bin_str not in meas_histogram:
            meas_histogram.update({bin_str: 1})
        else:
            meas_histogram[bin_str] += 1
        num_shots += 1
for k in meas_histogram:
    meas_histogram[k] /= num_shots

Next, read the qasm file, attatching the declaration of classical registers and measurement parts.
Using `cirq` library, run the noiseless qasm source simulation with the same shots with the experiment.

In [12]:
with open(os.path.join(ex_dir, ex_qasm)) as of:
    total_line = "".join(of.readlines())
pos = total_line.find(f"qreg q[{ex_params['n']}];\n")
prefix = f"creg meas[{ex_params['n']}];\n"
postfix = '\n'.join([f"measure q[{i}] -> meas[{i}];" for i in range(ex_params['n'])])
total_qasm = total_line[:pos] + prefix + total_line[pos:] + postfix
circuit = circuit_from_qasm(total_qasm)
simulator = Simulator()
result = simulator.run(circuit, repetitions=num_shots)

From the result, generate the histogram, simulated in noiseless environment.

In [13]:
ideal_histogram = dict()
key_pat = r'meas_(\d+)'
sorted_key = sorted([k for k in result.measurements], key=lambda x: int(re.match(key_pat, x).group(1)))
ideal_bin = zip(*[[str(x) for x in list(chain(*result.measurements[key]))]\
                      for key in sorted_key])
ideal_bin = ["".join(x) for x in ideal_bin]
for bin_str in ideal_bin:
    if bin_str not in ideal_histogram:
        ideal_histogram.update({bin_str: 1})
    else:
        ideal_histogram[bin_str] += 1
for k in ideal_histogram:
    ideal_histogram[k] /= num_shots

With experimental and simulated histograms, calculate the fidelity.

In [14]:
fid = get_xeb_from_hist_large(meas_histogram, ideal_histogram, ex_params['n'])
print(fid)

0.2652907069438546


Get the fidelity file from the dataset to compare it to the fidelity we calculated.
(They are characterized as 'patch', 'elided', 'full' and 'prediction', which we didn't understand the meaning. So we print them all.)

In [15]:
their_fidelity_path = './dataset/fidelity_4a.csv'
value = 0
with open(their_fidelity_path, 'r') as of:
    csv_reader = csv.reader(of, delimiter=',')
    for row in csv_reader:
        if row[0] == str(ex_params['n']):
            value = row
            break
    else:
        print("Fidelity not found.")
        raise ValueError
print(value[1:])

['0.2679', '0.292', '0.272', '0.2828']


So we see that the calculated fidelity and that from dataset is similar.